In [5]:
import dask
import dask.dataframe as dd
from azure.storage.blob import BlobServiceClient
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import os
import logging
import smtplib
from email.mime.text import MIMEText
from dotenv import load_dotenv

c:\Users\oluwa\anaconda3\envs\mygotoenv\lib\site-packages\snowflake\sqlalchemy\base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


In [1]:
import dask
import dask.dataframe as dd

In [3]:
from dotenv import load_dotenv

In [50]:
import snowflake.connector

In [3]:
import pandas as pd

In [4]:
load_dotenv(override=True)

True

In [3]:
# Logging configuration
logging.basicConfig(filename='etl_pipeline.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [17]:
# Configure the logging to be less detailed and add extra space between logs

logging.basicConfig(
    filename='ETL_pipeline.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s\n\n'  # Adjust the format as needed
)

In [ ]:
import logging

# Configure the logging to be less detailed and add extra space between logs
logging.basicConfig(
    filename='ETL_pipeline.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s\n\n'  # Adjust the format as needed
)

# Example usage
logging.info('This is an informational message.')
logging.error('This is an error message.')

In [ ]:
''' account_name = os.getenv('AZURE_ACCOUNT_NAME')
account_key = os.getenv('AZURE_ACCOUNT_KEY')
smtp_server = os.getenv('SMTP_SERVER')
smtp_port = int(os.getenv('SMTP_PORT'))
email_sender = os.getenv('EMAIL_SENDER')
email_recipient = os.getenv('EMAIL_RECIPIENT') '''

In [14]:
# ADLS BlobServiceClient

account_name = os.getenv('AzureAN')
account_key = os.getenv('AzureAK')
container_name = os.getenv('AzureCN')

blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)
container_client = blob_service_client.get_container_client(container_name)

In [20]:
# Extracting the Data and moving to another folder

def move_blob(blob_name, destination_folder):
    try:
        logging.info(f'Moving {blob_name} to {destination_folder}.')
        
        # Create the destination blob path
        destination_blob_name = f'{"payroll-extracteddata"}/{blob_name.split("/")[-1]}'  # Placeholder: 'destination_folder' should be the folder name where you want to move processed files
        
        # Get the source and destination blob clients
        source_blob = container_client.get_blob_client(blob_name)
        destination_blob = container_client.get_blob_client(destination_blob_name)

        # Copy the blob to the destination
        destination_blob.start_copy_from_url(source_blob.url)
        
        # Delete the original blob
        source_blob.delete_blob()

        logging.info(f'Moved {blob_name} to {destination_folder}.')
    except Exception as e:
        logging.error(f'Failed to move {blob_name}: {e}')



def extract_files_from_adls(destination_folder):
    try:
        # List blobs with a specific prefix (Placeholder: 'new-data/' should be replaced with the prefix of your new data folder)
        blobs_list = container_client.list_blobs(name_starts_with="payroll-sourcedata/")
        
        for blob in blobs_list:
            blob_name = blob.name
            
            # Define the local download path
            download_path = os.path.join("Raw_payroll_Data", blob_name.split('/')[-1])  # Placeholder: 'destination_folder' should be the local directory where files will be downloaded
            
            logging.info(f'Downloading {blob_name} from ADLS to {download_path}.')
            
            # Download the blob to the local path
            blob_client = container_client.get_blob_client(blob_name)
            with open(download_path, "wb") as file:
                file.write(blob_client.download_blob().readall())
                
            logging.info(f'Downloaded {blob_name} successfully.')
            
            # Move the blob to the processed-data folder after downloading (Placeholder: 'processed-data' should be the folder where you want to move processed files)
            move_blob(blob_name, 'payroll-extracteddata')
            
    except Exception as e:
        logging.error(f'Error extracting files from ADLS: {e}')


extract_files_from_adls('Raw_payroll_Data')


In [15]:
# To check the directory/folders in the specified container

blobs_list = container_client.list_blobs()

print(f'Found blobs: {[blob.name for blob in blobs_list]}')

Found blobs: ['payroll-extracteddata', 'payroll-sourcedata', 'payroll-sourcedata/nycpayroll_2020.csv']


In [4]:
# Converting the data to dask dataframe

NYCpayroll_df = dd.read_csv(r"Raw_payroll_Data\nycpayroll_2020.csv", infer_objects=True)

TypeError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: read_csv() got an unexpected keyword argument 'infer_objects'

In [31]:
nycpayroll_df = dd.read_csv('nycpayroll.csv', infer_objects=True)



FileNotFoundError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: [WinError 2] The system cannot find the file specified: 'c:/Users/oluwa/OneDrive/Documents/Data Engineering Training - 10Alytics/Capstone Project/NYC_Payroll_Pipeline/nycpayroll.csv'

In [24]:
# Data Transformation


NYCpayroll_df.info()

<class 'dask_expr.DataFrame'>
Columns: 19 entries, FiscalYear to TotalOtherPay
dtypes: float64(3), int64(8), string(8)

In [25]:
NYCpayroll_df.head()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+---------+----------+
| Column       | Found   | Expected |
+--------------+---------+----------+
| BaseSalary   | float64 | int64    |
| OTHours      | float64 | int64    |
| RegularHours | float64 | int64    |
+--------------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'BaseSalary': 'float64',
       'OTHours': 'float64',
       'RegularHours': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

In [5]:
NYCpayroll_df = dd.read_csv(r"Raw_payroll_Data\nycpayroll_2020.csv")

In [6]:
NYCpayroll_df.head()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+---------+----------+
| Column       | Found   | Expected |
+--------------+---------+----------+
| BaseSalary   | float64 | int64    |
| OTHours      | float64 | int64    |
| RegularHours | float64 | int64    |
+--------------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'BaseSalary': 'float64',
       'OTHours': 'float64',
       'RegularHours': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

In [7]:
dtypes = {
    "FiscalYear" : "int64",
    "PayrollNumber" : "int64",
    "AgencyID" : "int64",
    "AgencyName" : "object",
    "EmployeeID" : "int64",
    "LastName" : "object",
    "FirstName"  : "object",
    "AgencyStartDate" : "object",
    "WorkLocationBorough" : "object", 
    "TitleCode" : "int64",
   "TitleDescription" : "object",
   "LeaveStatusasofJune30" : "object",
   "BaseSalary" : "float64",
   "PayBasis" : "object" ,
  "RegularHours" : "float64",
   "RegularGrossPaid" : "float64",
   "OTHours" : "float64",
   "TotalOTPaid" : "float64",
   "TotalOtherPay" : "float64"
}

NYCpayroll_df = dd.read_csv(r"Dataset\Raw_payroll_Data\nycpayroll_2020.csv", dtype=dtypes)



In [8]:
NYCpayroll_df.head()

,FiscalYear,PayrollNumber,AgencyID,AgencyName,EmployeeID,LastName,FirstName,AgencyStartDate,WorkLocationBorough,TitleCode,TitleDescription,LeaveStatusasofJune30,BaseSalary,PayBasis,RegularHours,RegularGrossPaid,OTHours,TotalOTPaid,TotalOtherPay
0,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,10001,GEAGER,VERONICA,9/12/2016,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
1,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,149612,ROTTA,JONATHAN,9/16/2013,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
2,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,206583,WILSON II,ROBERT,4/30/2018,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
3,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,199874,WASHINGTON,MORIAH,3/18/2019,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,87900.95,0.0,0.0,-3202.74
4,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,58036,KRAWCZYK,AMANDA,5/15/2017,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,83976.54,0.0,0.0,0.00


In [10]:
NYCpayroll_df.info()

<class 'dask_expr.DataFrame'>
Columns: 19 entries, FiscalYear to TotalOtherPay
dtypes: float64(6), int64(5), string(8)

In [12]:
NYCpayroll_df.describe().compute()

,FiscalYear,PayrollNumber,AgencyID,EmployeeID,TitleCode,BaseSalary,RegularHours,RegularGrossPaid,OTHours,TotalOTPaid,TotalOtherPay
count,100.00,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,2019.78,17.260000,2120.260000,127735.740000,40407.590000,94344.419900,1164.937500,61417.815700,72.780000,3355.041500,5342.134200
std,2.20,0.675995,0.675995,56164.043495,95.128313,55375.949822,713.239513,57858.174131,109.869098,5006.887007,22425.288045
min,1998.00,17.000000,2120.000000,10001.000000,40081.000000,28.840000,0.000000,0.000000,0.000000,0.000000,-3202.740000
25%,2020.00,17.000000,2120.000000,92662.750000,40447.000000,66603.000000,385.000000,21620.255000,0.000000,0.000000,0.000000
50%,2020.00,17.000000,2120.000000,136026.500000,40448.000000,73765.500000,1431.500000,56058.285000,1.000000,62.510000,223.210000
75%,2020.00,17.000000,2120.000000,176790.000000,40448.000000,90361.250000,1820.000000,70199.420000,124.937500,5417.012500,1323.625000
max,2020.00,19.000000,2122.000000,212044.000000,40494.000000,243171.000000,1820.000000,239475.950000,526.000000,19751.620000,180605.850000


In [14]:
NYCpayroll_df.columns

Index(['FiscalYear', 'PayrollNumber', 'AgencyID', 'AgencyName', 'EmployeeID',
       'LastName', 'FirstName', 'AgencyStartDate', 'WorkLocationBorough',
       'TitleCode', 'TitleDescription', 'LeaveStatusasofJune30', 'BaseSalary',
       'PayBasis', 'RegularHours', 'RegularGrossPaid', 'OTHours',
       'TotalOTPaid', 'TotalOtherPay'],
      dtype='object')

In [16]:
# To check for null values and count the number of rows in each column 

NYCpayroll_df.isnull().compute().count()

FiscalYear               100
PayrollNumber            100
AgencyID                 100
AgencyName               100
EmployeeID               100
LastName                 100
FirstName                100
AgencyStartDate          100
WorkLocationBorough      100
TitleCode                100
TitleDescription         100
LeaveStatusasofJune30    100
BaseSalary               100
PayBasis                 100
RegularHours             100
RegularGrossPaid         100
OTHours                  100
TotalOTPaid              100
TotalOtherPay            100
dtype: int64

In [29]:
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler("etl_pipeline.log"),  # Logs to a file named transformation.log
        logging.StreamHandler()  # Also prints logs to the console
    ]
)

def transform_data(input_file_path, output_file_path): 
    try:
        logging.info("Starting data transformation.")

        # Define data types for columns
        dtypes = {
            "FiscalYear" : "int64",
            "PayrollNumber" : "int64",
            "AgencyID" : "int64",
            "AgencyName": "object",
            "EmployeeID": "int64",
            "LastName": "object",
            "FirstName": "object",
            "AgencyStartDate": "object",
            "WorkLocationBorough": "object",
            "TitleCode": "int64",
            "TitleDescription": "object",
            "LeaveStatusasofJune30": "object",
            "BaseSalary": "float64",
            "PayBasis": "object",
            "RegularHours": "float64",
            "RegularGrossPaid": "float64",
            "OTHours": "float64",
            "TotalOTPaid": "float64",
            "TotalOtherPay": "float64"
        }

        # Load the data
        NYCpayroll_df = dd.read_csv(r"Dataset\Raw_payroll_Data\nycpayroll_2020.csv", dtype=dtypes)

        # Drop duplicates
        NYCpayroll_df = NYCpayroll_df.drop_duplicates()

        # Convert date columns to datetime
        NYCpayroll_df['AgencyStartDate'] = dd.to_datetime(NYCpayroll_df['AgencyStartDate'], errors='coerce')

        # Rename columns
        NYCpayroll_df = NYCpayroll_df.rename(columns={'AgencyCode': 'AgencyID'})

        # Fill missing values
        NYCpayroll_df = NYCpayroll_df.fillna({"FiscalYear": 0,
                                                "PayrollNumber": 0,
                                                "AgencyID": 0,
                                                "AgencyName": "Unknown",
                                                "EmployeeID": 0,
                                                "LastName": "Unknown",
                                                "FirstName": "Unknown",
                                                "AgencyStartDate": pd.Timestamp('1900-01-01'),  # Default date for missing values
                                                "WorkLocationBorough": "Unknown",
                                                "TitleCode": 0,
                                                "TitleDescription": "Unknown",
                                                "LeaveStatusasofJune30": "Unknown",
                                                "BaseSalary": NYCpayroll_df["BaseSalary"].mean(),
                                                "PayBasis": "Unknown",
                                                "RegularHours": NYCpayroll_df["RegularHours"].mean(),
                                                "RegularGrossPaid": NYCpayroll_df["RegularGrossPaid"].mean(),
                                                "OTHours": NYCpayroll_df["OTHours"].mean(),
                                                "TotalOTPaid": NYCpayroll_df["TotalOTPaid"].mean(),
                                                "TotalOtherPay": NYCpayroll_df["TotalOtherPay"].mean()
                                            })

        logging.info("Data transformation completed successfully.")

        # converting dask to pandas to enable loading to snowflakes
        NYCpayroll_df = NYCpayroll_df.compute() 
        # Save the transformed data back to disk
        NYCpayroll_df.to_csv(output_file_path, index=False)
        logging.info(f"Transformed data saved to {output_file_path}")

    except Exception as e:
        logging.error(f"Error during data transformation: {e}")
        raise

    return NYCpayroll_df

    # Trigger the transformation
transform_data(
    r"Dataset/Raw_payroll_Data/nycpayroll_2020.csv", 
    r"Dataset/Cleaned_payroll_Data\nycpayroll_2020.csv")



2024-08-24 01:45:57,647  INFO  Starting data transformation.


2024-08-24 01:45:57,742  INFO  Data transformation completed successfully.
2024-08-24 01:45:57,889  INFO  Transformed data saved to Dataset/Cleaned_payroll_Data\nycpayroll_2020.csv


,FiscalYear,PayrollNumber,AgencyID,AgencyName,EmployeeID,LastName,FirstName,AgencyStartDate,WorkLocationBorough,TitleCode,TitleDescription,LeaveStatusasofJune30,BaseSalary,PayBasis,RegularHours,RegularGrossPaid,OTHours,TotalOTPaid,TotalOtherPay
0,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,10001,GEAGER,VERONICA,2016-09-12,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
1,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,149612,ROTTA,JONATHAN,2013-09-16,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
2,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,206583,WILSON II,ROBERT,2018-04-30,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
3,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,199874,WASHINGTON,MORIAH,2019-03-18,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,87900.95,0.0,0.0,-3202.74
4,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,58036,KRAWCZYK,AMANDA,2017-05-15,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,83976.54,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020,19,2122,OFFICE OF MANAGEMENT & BUDGET,33054,HIGGINS,JAMES,1982-06-21,MANHATTAN,40306,COMPUTER SYSTEMS MANAGER,ACTIVE,195826.0,per Annum,1820.0,192850.37,0.0,0.0,0.00
96,2020,19,2122,OFFICE OF MANAGEMENT & BUDGET,19997,GREENBERG,DAVID,2008-07-14,MANHATTAN,40199,BUDGET ANALYST,ACTIVE,196950.0,per Annum,1820.0,192107.94,0.0,0.0,500.00
97,2020,19,2122,OFFICE OF MANAGEMENT & BUDGET,61906,LARSON,CHRISTINE,2005-12-25,MANHATTAN,40199,BUDGET ANALYST,ACTIVE,185843.0,per Annum,1820.0,183019.14,0.0,0.0,0.00
98,2020,19,2122,OFFICE OF MANAGEMENT & BUDGET,19003,GRATHWOL,JOHN,1988-09-27,MANHATTAN,40199,BUDGET ANALYST,CEASED,216431.0,per Annum,0.0,0.00,0.0,0.0,180605.85


In [ ]:
# transform.py

import dask.dataframe as dd
import pandas as pd
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("logs/transformation.log"),
        logging.StreamHandler()
    ]
)

def transform_data(input_file_path):
    """
    Transforms raw NYC payroll data.

    Parameters:
        input_file_path (str): Path to the raw input CSV file.

    Returns:
        dask.dataframe.DataFrame: Transformed Dask DataFrame.
    """
    try:
        logging.info("Starting data transformation.")

        # Define data types for columns
        dtypes = {
            "FiscalYear": "int64",
            "PayrollNumber": "int64",
            "AgencyID": "int64",
            "AgencyName": "object",
            "EmployeeID": "int64",
            "LastName": "object",
            "FirstName": "object",
            "AgencyStartDate": "object",
            "WorkLocationBorough": "object",
            "TitleCode": "int64",
            "TitleDescription": "object",
            "LeaveStatusasofJune30": "object",
            "BaseSalary": "float64",
            "PayBasis": "object",
            "RegularHours": "float64",
            "RegularGrossPaid": "float64",
            "OTHours": "float64",
            "TotalOTPaid": "float64",
            "TotalOtherPay": "float64"
        }

        # Load data into a Dask DataFrame
        logging.info(f"Loading data from {"Raw_payroll_Data\nycpayroll_2020.csv"}")
        NYCpayroll_df = dd.read_csv(r"Raw_payroll_Data\nycpayroll_2020.csv", dtype=dtypes)
        df = dd.read_csv(input_file_path, dtype=dtypes, assume_missing=True)

        # Drop duplicates
        df = df.drop_duplicates()
        logging.info("Dropped duplicate rows.")

        # Convert 'AgencyStartDate' to datetime
        df['AgencyStartDate'] = dd.to_datetime(df['AgencyStartDate'], errors='coerce')
        logging.info("Converted 'AgencyStartDate' to datetime.")

        # Rename columns if necessary
        # Example: df = df.rename(columns={'OldColumnName': 'NewColumnName'})
        # logging.info("Renamed columns.")

        # Calculate mean values for numerical columns to fill missing data
        base_salary_mean = df['BaseSalary'].mean().compute()
        regular_hours_mean = df['RegularHours'].mean().compute()
        regular_gross_paid_mean = df['RegularGrossPaid'].mean().compute()
        ot_hours_mean = df['OTHours'].mean().compute()
        total_ot_paid_mean = df['TotalOTPaid'].mean().compute()
        total_other_pay_mean = df['TotalOtherPay'].mean().compute()

        # Fill missing values
        df = df.fillna({
            "FiscalYear": 0,
            "PayrollNumber": 0,
            "AgencyID": 0,
            "AgencyName": "Unknown",
            "EmployeeID": 0,
            "LastName": "Unknown",
            "FirstName": "Unknown",
            "AgencyStartDate": pd.Timestamp('1900-01-01'),
            "WorkLocationBorough": "Unknown",
            "TitleCode": 0,
            "TitleDescription": "Unknown",
            "LeaveStatusasofJune30": "Unknown",
            "BaseSalary": base_salary_mean,
            "PayBasis": "Unknown",
            "RegularHours": regular_hours_mean,
            "RegularGrossPaid": regular_gross_paid_mean,
            "OTHours": ot_hours_mean,
            "TotalOTPaid": total_ot_paid_mean,
            "TotalOtherPay": total_other_pay_mean
        })
        logging.info("Filled missing values.")

        logging.info("Data transformation completed successfully.")
        return df

    except Exception as e:
        logging.error(f"An error occurred during transformation: {e}")
        raise


In [24]:
NYCpayroll_df.columns

Index(['FiscalYear', 'PayrollNumber', 'AgencyID', 'AgencyName', 'EmployeeID',
       'LastName', 'FirstName', 'AgencyStartDate', 'WorkLocationBorough',
       'TitleCode', 'TitleDescription', 'LeaveStatusasofJune30', 'BaseSalary',
       'PayBasis', 'RegularHours', 'RegularGrossPaid', 'OTHours',
       'TotalOTPaid', 'TotalOtherPay'],
      dtype='object')

In [26]:
NYCpayroll_df.info

<bound method DataFrame.info of Dask DataFrame Structure:
              FiscalYear PayrollNumber AgencyID AgencyName EmployeeID LastName FirstName AgencyStartDate WorkLocationBorough TitleCode TitleDescription LeaveStatusasofJune30 BaseSalary PayBasis RegularHours RegularGrossPaid  OTHours TotalOTPaid TotalOtherPay
npartitions=1                                                                                                                                                                                                                                                     
                   int64         int64    int64     string      int64   string    string          string              string     int64           string                string    float64   string      float64          float64  float64     float64       float64
                     ...           ...      ...        ...        ...      ...       ...             ...                 ...       ...              ...              

In [27]:
# Transformation 1: Generate minimal dim_date based on FiscalYear
def generate_dim_date(fiscal_years):
    dim_date = pd.DataFrame({
        'DateKey': [int(f"{year}0101") for year in fiscal_years],
        'Date': [pd.Timestamp(f"{year}-01-01") for year in fiscal_years],
        'Year': fiscal_years,
        'Month': [1] * len(fiscal_years),
        'Day': [1] * len(fiscal_years),
        'Quarter': [1] * len(fiscal_years)
    })
    return dim_date

# Extract distinct fiscal years
fiscal_years = NYCpayroll_df['FiscalYear'].drop_duplicates().compute().tolist()

# Generate dim_date DataFrame
dim_date_df = generate_dim_date(fiscal_years)



In [29]:
dim_date_df.head()

,DateKey,Date,Year,Month,Day,Quarter
0,20200101,2020-01-01,2020,1,1,1
1,19980101,1998-01-01,1998,1,1,1


In [ ]:
# Save dim_date_df to CSV (for loading into Snowflake later)
dim_date_output_path = os.path.join("<path_to_save_dim_date>", "dim_date.csv")
dim_date_df.to_csv(dim_date_output_path, index=False)


In [32]:
print(NYCpayroll_df.dtypes)


FiscalYear                         int64
PayrollNumber                      int64
AgencyID                           int64
AgencyName               string[pyarrow]
EmployeeID                         int64
LastName                 string[pyarrow]
FirstName                string[pyarrow]
AgencyStartDate          string[pyarrow]
WorkLocationBorough      string[pyarrow]
TitleCode                          int64
TitleDescription         string[pyarrow]
LeaveStatusasofJune30    string[pyarrow]
BaseSalary                       float64
PayBasis                 string[pyarrow]
RegularHours                     float64
RegularGrossPaid                 float64
OTHours                          float64
TotalOTPaid                      float64
TotalOtherPay                    float64
dtype: object


In [23]:
print(type(FiscalYear))

<class 'dask_expr._collection.Series'>


In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

# File paths (placeholders)
input_file_path = "<path_to_input_file>"
output_file_path = "<path_to_output_file>"

# Load the CSV file as a Dask DataFrame
NYCpayroll_df = dd.read_csv(input_file_path)

# Transformation 1: Generate minimal dim_date based on FiscalYear
def generate_dim_date(fiscal_years):
    dim_date = pd.DataFrame({
        'DateKey': [int(f"{year}0101") for year in fiscal_years],
        'Date': [pd.Timestamp(f"{year}-01-01") for year in fiscal_years],
        'Year': fiscal_years,
        'Month': [1] * len(fiscal_years),
        'Day': [1] * len(fiscal_years),
        'Quarter': [1] * len(fiscal_years)
    })
    return dim_date

# Extract distinct fiscal years
fiscal_years = NYCpayroll_df['FiscalYear'].drop_duplicates().compute().tolist()

# Generate dim_date DataFrame
dim_date_df = generate_dim_date(fiscal_years)

# Save dim_date_df to CSV (for loading into Snowflake later)
dim_date_output_path = os.path.join("<path_to_save_dim_date>", "dim_date.csv")
dim_date_df.to_csv(dim_date_output_path, index=False)

# Transformation 2: Add PayBasisID and PayBasisDescription columns
def map_paybasis(df):
    # Generate unique IDs for PayBasisDescription
    df['PayBasisID'] = df['PayBasisDescription'].factorize()[0] + 1
    return df

# Apply the mapping function
NYCpayroll_df = NYCpayroll_df.map_partitions(map_paybasis)

# Other transformations (e.g., dropping duplicates, handling missing values, renaming columns)
NYCpayroll_df = NYCpayroll_df.drop_duplicates()
NYCpayroll_df['AgencyStartDate'] = dd.to_datetime(NYCpayroll_df['AgencyStartDate'], errors='coerce')
NYCpayroll_df = NYCpayroll_df.rename(columns={'AgencyCode': 'AgencyID'})

# Save the transformed data to CSV
NYCpayroll_df.to_csv(output_file_path, index=False, single_file=True)

print("Transformation complete and files saved.")


In [6]:
def transform_data(NYCpayroll_df):
    try:
            
        dtypes = {
        "FiscalYear" : "int64",
        "PayrollNumber" : "int64",
        "AgencyID" : "int64",
        "AgencyName" : "object",
        "EmployeeID" : "int64",
        "LastName" : "object",
        "FirstName"  : "object",
        "AgencyStartDate" : "object",
        "WorkLocationBorough" : "object", 
        "TitleCode" : "int64",
        "TitleDescription" : "object",
        "LeaveStatusasofJune30" : "object",
        "BaseSalary" : "float64",
        "PayBasis" : "object" ,
        "RegularHours" : "float64",
        "RegularGrossPaid" : "float64",
        "OTHours" : "float64",
        "TotalOTPaid" : "float64",
        "TotalOtherPay" : "float64"
    }
        NYCpayroll_df = dd.read_csv(r"Raw_payroll_Data\nycpayroll_2020.csv", dtype=dtypes)

        # Drop duplicates

        NYCpayroll_df = NYCpayroll_df.drop_duplicates()

        # Convert date columns to datetime
        NYCpayroll_df['AgencyStartDate'] = dd.to_datetime(NYCpayroll_df['AgencyStartDate'], errors='coerce', inplace=True)

        # Rename columns
        NYCpayroll_df = NYCpayroll_df.rename(columns={'AgencyCode': 'AgencyID'}, inplace=True)


        # Fill missing values
        NYCpayroll_df = NYCpayroll_df.fillna ({"FiscalYear" : 0000,
                                                "PayrollNumber" : 00,
                                                "AgencyID" : 0000,
                                                "AgencyName" : "Unknown",
                                                "EmployeeID" : 000000,
                                                "LastName" : "Unknown",
                                                "FirstName"  : "Unknown",
                                                "AgencyStartDate" : pd.Timestamp('0000/00/00'),
                                                "WorkLocationBorough" : "Unknown", 
                                                "TitleCode" : 00000,
                                                "TitleDescription" : "Unknown",
                                                "LeaveStatusasofJune30" : "Unknown",
                                                "BaseSalary" : NYCpayroll_df["BaseSalary"].mean(),
                                                "PayBasis" : "Unknown",
                                                "RegularHours" : NYCpayroll_df["BaseSalary"].mean(),
                                                "RegularGrossPaid" : NYCpayroll_df["BaseSalary"].mean(),
                                                "OTHours" : NYCpayroll_df["BaseSalary"].mean(),
                                                "TotalOTPaid" : NYCpayroll_df["BaseSalary"].mean(),
                                                "TotalOtherPay" : NYCpayroll_df["BaseSalary"].mean()
                                            },   inplace=True)

        logging.info("Data transformation completed successfully.")
        

    except Exception as e:
        logging.error(f"Error during data transformation: {e}")
        raise

    return NYCpayroll_df


In [16]:
import os

In [ ]:
import dask.dataframe as dd
import pandas as pd
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler("transformation.log"),  # Logs to a file named transformation.log
        logging.StreamHandler()  # Also prints logs to the console
    ]
)

def transform_data(input_file_path, output_file_path):
    try:
        logging.info("Starting data transformation.")

        # Define data types for columns
        dtypes = {
            "FiscalYear": "int64",
            "PayrollNumber": "int64",
            "AgencyID": "int64",
            "AgencyName": "object",
            "EmployeeID": "int64",
            "LastName": "object",
            "FirstName": "object",
            "AgencyStartDate": "object",
            "WorkLocationBorough": "object",
            "TitleCode": "int64",
            "TitleDescription": "object",
            "LeaveStatusasofJune30": "object",
            "BaseSalary": "float64",
            "PayBasis": "object",
            "RegularHours": "float64",
            "RegularGrossPaid": "float64",
            "OTHours": "float64",
            "TotalOTPaid": "float64",
            "TotalOtherPay": "float64"
        }

        # Load the data
        NYCpayroll_df = dd.read_csv(input_file_path, dtype=dtypes)

        # Drop duplicates
        NYCpayroll_df = NYCpayroll_df.drop_duplicates()

        # Convert date columns to datetime
        NYCpayroll_df['AgencyStartDate'] = dd.to_datetime(NYCpayroll_df['AgencyStartDate'], errors='coerce')

        # Rename columns
        NYCpayroll_df = NYCpayroll_df.rename(columns={'AgencyCode': 'AgencyID'})

        # Fill missing values
        NYCpayroll_df = NYCpayroll_df.fillna({
            "FiscalYear": 0,
            "PayrollNumber": 0,
            "AgencyID": 0,
            "AgencyName": "Unknown",
            "EmployeeID": 0,
            "LastName": "Unknown",
            "FirstName": "Unknown",
            "AgencyStartDate": pd.Timestamp('1900-01-01'),  # Default date for missing values
            "WorkLocationBorough": "Unknown",
            "TitleCode": 0,
            "TitleDescription": "Unknown",
            "LeaveStatusasofJune30": "Unknown",
            "BaseSalary": NYCpayroll_df["BaseSalary"].mean(),
            "PayBasis": "Unknown",
            "RegularHours": NYCpayroll_df["RegularHours"].mean(),
            "RegularGrossPaid": NYCpayroll_df["RegularGrossPaid"].mean(),
            "OTHours": NYCpayroll_df["OTHours"].mean(),
            "TotalOTPaid": NYCpayroll_df["TotalOTPaid"].mean(),
            "TotalOtherPay": NYCpayroll_df["TotalOtherPay"].mean()
        })

        logging.info("Data transformation completed successfully.")

        # Save the transformed data back to disk
        NYCpayroll_df.to_csv(output_file_path, index=False, single_file=True)
        logging.info(f"Transformed data saved to {output_file_path}")

    except Exception as e:
        logging.error(f"Error during data transformation: {e}")
        raise

    return NYCpayroll_df

if __name__ == "__main__":
    # Placeholder: Replace with actual paths
    input_file_path = r"Raw_payroll_Data/nycpayroll_2020.csv"
    output_file_path = r"Transformed_payroll_Data/nycpayroll_2020_transformed.csv"

    # Trigger the transformation
    transform_data(input_file_path, output_file_path)


In [ ]:


'''logging.basicConfig(
    filename='ETL_pipeline.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s\n\n'  # Adjust the format as needed
)
'''
#format = ('%(asctime)s' 
    #         '%(levelname)s'  
    #         '%(message)s\n\n'#
   # ),


logging.basicConfig(
    filename="running_pipeline.log",
    level=logging.INFO,
    format = '%(asctime)s  %(levelname)s  %(message)s\n\n',
    handlers=[
        logging.FileHandler("running_pipeline.log"),  # Logs to a file named transformation.log
        logging.StreamHandler()  # Also prints logs to the console
    ]
)

#### Loading Data

In [33]:
print(NYCpayroll_df.dtypes)


FiscalYear                         int64
PayrollNumber                      int64
AgencyID                           int64
AgencyName               string[pyarrow]
EmployeeID                         int64
LastName                 string[pyarrow]
FirstName                string[pyarrow]
AgencyStartDate          string[pyarrow]
WorkLocationBorough      string[pyarrow]
TitleCode                          int64
TitleDescription         string[pyarrow]
LeaveStatusasofJune30    string[pyarrow]
BaseSalary                       float64
PayBasis                 string[pyarrow]
RegularHours                     float64
RegularGrossPaid                 float64
OTHours                          float64
TotalOTPaid                      float64
TotalOtherPay                    float64
dtype: object


In [34]:
print(NYCpayroll_df.columns)


Index(['FiscalYear', 'PayrollNumber', 'AgencyID', 'AgencyName', 'EmployeeID',
       'LastName', 'FirstName', 'AgencyStartDate', 'WorkLocationBorough',
       'TitleCode', 'TitleDescription', 'LeaveStatusasofJune30', 'BaseSalary',
       'PayBasis', 'RegularHours', 'RegularGrossPaid', 'OTHours',
       'TotalOTPaid', 'TotalOtherPay'],
      dtype='object')


In [ ]:
# Create the Tables:

# Dim_Date_Table



In [38]:
# Dim_Employee_Table

Employee = NYCpayroll_df[['EmployeeID', 'LastName', 'FirstName', 'AgencyStartDate', 'AgencyID', 'WorkLocationBorough','TitleCode',\
                          'TitleDescription', 'LeaveStatusasofJune30']].copy().drop_duplicates().reset_index(drop=True)

In [39]:
Employee.head()

,EmployeeID,LastName,FirstName,AgencyStartDate,AgencyID,WorkLocationBorough,TitleCode,TitleDescription,LeaveStatusasofJune30
0,10001,GEAGER,VERONICA,9/12/2016,2120,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE
1,149612,ROTTA,JONATHAN,9/16/2013,2120,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE
2,206583,WILSON II,ROBERT,4/30/2018,2120,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE
3,199874,WASHINGTON,MORIAH,3/18/2019,2120,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE
4,58036,KRAWCZYK,AMANDA,5/15/2017,2120,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE


In [40]:
# Dim_Agency_Table

Agency = NYCpayroll_df[['AgencyID', 'AgencyName']].copy().drop_duplicates().reset_index(drop=True)

In [41]:
Agency.head()

,AgencyID,AgencyName
0,2120,OFFICE OF EMERGENCY MANAGEMENT
1,2122,OFFICE OF MANAGEMENT & BUDGET


In [42]:
# Dim_PayBasis_Table

PayBasis = NYCpayroll_df[['PayBasis']]
PayBasis['PayBasisID'] = range(1, len(PayBasis) + 1)
PayBasis = PayBasis[['PayBasisID', 'PayBasis']]

In [49]:
PayBasis.info()

<class 'dask_expr.DataFrame'>
Columns: 2 entries, PayBasisID to PayBasis
dtypes: int64(1), string(1)

In [ ]:
# NYC_Payroll_Fact _Table

Fact = [['FiscalYear', 'FactID' 'AgencyID', 'PayBasisID', 'EmployeeID', 'PayrollNumber', 'BaseSalary','RegularHours', 'RegularGrossPaid',\
          'OTHours', 'TotalOTPaid', 'TotalOtherPay']]

In [52]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [53]:
print(conn_params)

NameError: name 'conn_params' is not defined

In [3]:
from load import dim_date_df, transformed_nycpayroll_df

c:\Users\oluwa\anaconda3\envs\mygotoenv\lib\site-packages\snowflake\sqlalchemy\base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


ImportError: cannot import name 'dim_date_df' from 'load' (c:\Users\oluwa\OneDrive\Documents\Data Engineering Training - 10Alytics\Project - Capstone\NYCPayroll_ETL\load.py)

In [2]:
dim_date_df.head()

NameError: name 'dim_date_df' is not defined

In [4]:






def load_data_to_snowflake(transformed_file_path, dim_date_file_path):

    try:

         # Initialize Dask client with 1 worker
        client = Client(n_workers=1)
        logging.info("Dask client initialized with 1 worker.")
        # Get Snowflake engine
        engine = get_snowflake_engine()


         # Define table structures
       
        tables = [
            {
                "name": "Dim_Employee",
                "columns": [
                    {"name": "EmployeeID", "type": "INT"},
                    {"name": "LastName", "type": "VARCHAR(50)"},
                    {"name": "FirstName", "type": "VARCHAR(50)"},
                    {"name": "AgencyStartDate", "type": "DATE"},
                    {"name": "TitleCode", "type": "INT"},
                    {"name": "TitleDescription", "type": "TEXT"},
                    {"name": "WorkLocationBorough", "type": "VARCHAR(255)"},
                    {"name": "LeaveStatusasofJune30", "type": "VARCHAR(50)"}
                ]
            },
            {
                "name": "Dim_Agency",
                "columns": [
                    {"name": "AgencyID", "type": "INT"},
                    {"name": "AgencyName", "type": "VARCHAR(255)"}
                ]
            },
            {
                "name": "Dim_PayBasis",
                "columns": [
                    {"name": "PayBasisID", "type": "INT"},
                    {"name": "PayBasis", "type": "VARCHAR(255)"}
                ]
            },
            {
                "name": "Dim_Date",
                "columns": [
                    {"name": "DateID", "type": "INT"},
                    {"name": "Date", "type": "DATE"},
                    {"name": "Year", "type": "INT"},
                    {"name": "Month", "type": "INT"},
                    {"name": "Day", "type": "INT"},
                    {"name": "Quarter", "type": "INT"}
                ]
            },
            {
                "name": "FactPayroll",
                "columns": [
                    {"name": "FactID", "type": "INT"},
                    {"name": "EmployeeID", "type": "INT"},
                    {"name": "AgencyID", "type": "INT"},
                    {"name": "DateID", "type": "INT"},
                    {"name": "PayBasisID", "type": "INT"},
                    {"name": "FiscalYear", "type": "INT"},
                    {"name": "PayrollNumber", "type": "INT"},
                    {"name": "BaseSalary", "type": "FLOAT"},
                    {"name": "RegularHours", "type": "FLOAT"},
                    {"name": "OTHours", "type": "FLOAT"},
                    {"name": "RegularGrossPaid", "type": "FLOAT"},
                    {"name": "TotalOTPaid", "type": "FLOAT"},
                    {"name": "TotalOtherPay", "type": "FLOAT"}
                ]
            }
        ]

        # Step 1: Load the main transformed CSV file
        transformed_nycpayroll_df = pd.read_csv("Dataset/Cleaned_payroll_Data/nycpayroll_2020.csv")
        logging.info("Transformed NycPayroll CSV file loaded successfully.")

        # Step 2: Load data into Snowflake

        with engine.connect() as conn:

            # Step 3: Truncate the tables before loading new data
            for table in tables:
                table_name = table["name"]
                columns = table["columns"]
                column_defs = ', '.join([f'"{col["name"]}" {col["type"]}' for col in columns])
                create_table_sql = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({column_defs});'
                conn.execute(text(create_table_sql))
                logging.info(f'Table "{table_name}" created or already exists.')

            # Step 2: Truncate the tables before loading new data
            for table in tables:
                truncate_table_sql = f'TRUNCATE TABLE "{table["name"]}";'
                conn.execute(text(truncate_table_sql))
                logging.info(f'Truncated "{table["name"]}" successfully.')

            # Dim_Employee
            Dim_Employee_df = transformed_nycpayroll_df[[
                "EmployeeID", "LastName", "FirstName", "AgencyStartDate", 
                "TitleCode", "TitleDescription", "WorkLocationBorough", 
                "LeaveStatusasofJune30"
            ]]
            Dim_Employee_df.to_sql("Dim_Employee", con=conn, if_exists="replace", index=False)
            logging.info('Dim_Employee data loaded successfully!!')

            # Dim_Agency
            Dim_Agency_df = transformed_nycpayroll_df[["AgencyID", "AgencyName"]]
            Dim_Agency_df.to_sql("Dim_Agency", con=conn, if_exists="replace", index=False)
            logging.info('Dim_Agency data loaded successfully!!')

            # Dim_PayBasis
            dim_paybasis_df = transformed_nycpayroll_df[["PayBasisID", "PayBasis"]]
            dim_paybasis_df.to_sql("Dim_PayBasis", con=conn, if_exists="replace", index=False)
            logging.info('Dim_PayBasis data loaded successfully!!')

            # Dim_Date (from dim_date.csv)
            dim_date_df = pd.read_csv("Dataset/Cleaned_payroll_Data/dim_date.csv")
            dim_date_df.to_sql("Dim_Date", con=conn, if_exists="replace", index=False)
            logging.info('Dim_Date data loaded successfully.')

            # FactPayroll
            # Merge with dim_date_df to include DateID
            factpayroll_df = transformed_nycpayroll_df.merge(dim_date_df, left_on="FiscalYear", right_on="Year", how="inner") \
                                                        [["EmployeeID", "AgencyID", "DateID", "PayBasisID", "FiscalYear", \
                                                        "PayrollNumber", "BaseSalary", "RegularHours", "OTHours", \
                                                        "RegularGrossPaid", "TotalOTPaid", "TotalOtherPay"]]

            # Creating the FactID after the merging
            factpayroll_df["FactID"] = range(1, len(factpayroll_df) + 1)

            # Reorder columns to have FactID first
            factpayroll_df = factpayroll_df[[
                "FactID", "EmployeeID", "AgencyID", "DateID", "PayBasisID", "FiscalYear", \
                "PayrollNumber", "BaseSalary", "RegularHours", "OTHours", "RegularGrossPaid", \
                "TotalOTPaid", "TotalOtherPay"
            ]]

            factpayroll_df.to_sql("FactPayroll", con=conn, if_exists="replace", index=False)
            logging.info('FactPayroll data loaded successfully!!')

            # Commit the changes
            logging.info("All data loaded successfully!!")

        # Close the Dask client
        client.close()
        logging.info("Dask client closed.")

    except Exception as e:
        logging.error(f"Error loading data into Snowflake: {e}")
        raise

In [6]:
import sqlalchemy
import snowflake.sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv
from pipeline_log import logging
from dask.distributed import Client
from sqlalchemy import text


In [ ]:
load_dotenv(override=True)

In [ ]:



# Define the Snowflake connection parameters
def get_snowflake_engine():
    try:
        engine = create_engine(
            "snowflake://{user}:{password}@{account_identifier}/{database}/{schema}?warehouse={warehouse}".format(
                user=os.getenv("sn_user"),
                password=os.getenv("sn_pword"),
                account_identifier=os.getenv("sn_Acct_Id"),
                database=os.getenv("sn_DB"),
                schema=os.getenv("sn_schema"),
                warehouse=os.getenv("sn_DWH")
            )
        )
        logging.info("Connected to Snowflake successfully.")
        return engine
    except Exception as e:
        logging.error(f"Failed to connect to Snowflake: {e}")
        raise

In [16]:

tables = [
            {
                "name": "Dim_Employee",
                "columns": [
                    {"name": "EmployeeID", "type": "INT"},
                    {"name": "LastName", "type": "VARCHAR(50)"},
                    {"name": "FirstName", "type": "VARCHAR(50)"},
                    {"name": "AgencyStartDate", "type": "DATE"},
                    {"name": "TitleCode", "type": "INT"},
                    {"name": "TitleDescription", "type": "TEXT"},
                    {"name": "WorkLocationBorough", "type": "VARCHAR(255)"},
                    {"name": "LeaveStatusasofJune30", "type": "VARCHAR(50)"}
                ]
            },
            {
                "name": "Dim_Agency",
                "columns": [
                    {"name": "AgencyID", "type": "INT"},
                    {"name": "AgencyName", "type": "VARCHAR(255)"}
                ]
            },
            {
                "name": "Dim_PayBasis",
                "columns": [
                    {"name": "PayBasisID", "type": "INT"},
                    {"name": "PayBasis", "type": "VARCHAR(255)"}
                ]
            },
            {
                "name": "Dim_Date",
                "columns": [
                    {"name": "DateID", "type": "INT"},
                    {"name": "Date", "type": "DATE"},
                    {"name": "Year", "type": "INT"},
                    {"name": "Month", "type": "INT"},
                    {"name": "Day", "type": "INT"},
                    {"name": "Quarter", "type": "INT"}
                ]
            },
            {
                "name": "FactPayroll",
                "columns": [
                    {"name": "FactID", "type": "INT"},
                    {"name": "EmployeeID", "type": "INT"},
                    {"name": "AgencyID", "type": "INT"},
                    {"name": "DateID", "type": "INT"},
                    {"name": "PayBasisID", "type": "INT"},
                    {"name": "FiscalYear", "type": "INT"},
                    {"name": "PayrollNumber", "type": "INT"},
                    {"name": "BaseSalary", "type": "FLOAT"},
                    {"name": "RegularHours", "type": "FLOAT"},
                    {"name": "OTHours", "type": "FLOAT"},
                    {"name": "RegularGrossPaid", "type": "FLOAT"},
                    {"name": "TotalOTPaid", "type": "FLOAT"},
                    {"name": "TotalOtherPay", "type": "FLOAT"}
                ]
            }
        ]

       


In [17]:
for table in tables:
                table_name = table["name"]
                columns = table["columns"]
                column_defs = ', '.join([f'"{col["name"]}" {col["type"]}' for col in columns])
                create_table_sql = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({column_defs});'

In [20]:
print(column_defs)

"FactID" INT, "EmployeeID" INT, "AgencyID" INT, "DateID" INT, "PayBasisID" INT, "FiscalYear" INT, "PayrollNumber" INT, "BaseSalary" FLOAT, "RegularHours" FLOAT, "OTHours" FLOAT, "RegularGrossPaid" FLOAT, "TotalOTPaid" FLOAT, "TotalOtherPay" FLOAT


In [7]:
 # Step 1: Load the main transformed CSV file
transformed_nycpayroll_df = pd.read_csv("Dataset/Cleaned_payroll_Data/nycpayroll_2020.csv")
logging.info("Transformed NycPayroll CSV file loaded successfully.")

2024-09-05 07:32:55,203 - INFO - Transformed NycPayroll CSV file loaded successfully.




In [8]:
# Dim_Employee
Dim_Employee_df = transformed_nycpayroll_df[[
                "EmployeeID", "LastName", "FirstName", "AgencyStartDate", 
                "TitleCode", "TitleDescription", "WorkLocationBorough", 
                "LeaveStatusasofJune30"
            ]]

Dim_Employee_df.head()

,EmployeeID,LastName,FirstName,AgencyStartDate,TitleCode,TitleDescription,WorkLocationBorough,LeaveStatusasofJune30
0,10001,GEAGER,VERONICA,2016-09-12,40447,EMERGENCY PREPAREDNESS MANAGER,BROOKLYN,ACTIVE
1,149612,ROTTA,JONATHAN,2013-09-16,40447,EMERGENCY PREPAREDNESS MANAGER,BROOKLYN,ACTIVE
2,206583,WILSON II,ROBERT,2018-04-30,40447,EMERGENCY PREPAREDNESS MANAGER,BROOKLYN,ACTIVE
3,199874,WASHINGTON,MORIAH,2019-03-18,40447,EMERGENCY PREPAREDNESS MANAGER,BROOKLYN,ACTIVE
4,58036,KRAWCZYK,AMANDA,2017-05-15,40447,EMERGENCY PREPAREDNESS MANAGER,BROOKLYN,ACTIVE


In [ ]:
Dim_Employee_df.to_sql("Dim_Employee", con=conn, if_exists="replace", index=False)
logging.info('Dim_Employee data loaded successfully!!')

In [9]:
 # Dim_Agency
Dim_Agency_df = transformed_nycpayroll_df[["AgencyID", "AgencyName"]]
Dim_Agency_df.head()

,AgencyID,AgencyName
0,2120,OFFICE OF EMERGENCY MANAGEMENT
1,2120,OFFICE OF EMERGENCY MANAGEMENT
2,2120,OFFICE OF EMERGENCY MANAGEMENT
3,2120,OFFICE OF EMERGENCY MANAGEMENT
4,2120,OFFICE OF EMERGENCY MANAGEMENT


In [ ]:
Dim_Agency_df.to_sql("Dim_Agency", con=conn, if_exists="replace", index=False)
logging.info('Dim_Agency data loaded successfully!!')


In [10]:
# Dim_PayBasis
dim_paybasis_df = transformed_nycpayroll_df[["PayBasisID", "PayBasis"]]
dim_paybasis_df.head()

,PayBasisID,PayBasis
0,1,per Annum
1,1,per Annum
2,1,per Annum
3,1,per Annum
4,1,per Annum


In [ ]:
 
dim_paybasis_df.to_sql("Dim_PayBasis", con=conn, if_exists="replace", index=False)
logging.info('Dim_PayBasis data loaded successfully!!')

In [11]:
# Dim_Date (from dim_date.csv)
dim_date_df = pd.read_csv("Dataset/Cleaned_payroll_Data/dim_date.csv")

dim_date_df.head()

,DateID,Date,Year,Month,Day,Quarter
0,20200000,2020-01-01,2020,1,1,1
1,19980000,1998-01-01,1998,1,1,1


In [ ]:
dim_date_df.to_sql("Dim_Date", con=conn, if_exists="replace", index=False)
logging.info('Dim_Date data loaded successfully.')


In [12]:
 # FactPayroll
# Merge with dim_date_df to include DateID
factpayroll_df = transformed_nycpayroll_df.merge(dim_date_df, left_on="FiscalYear", right_on="Year", how="inner") \
                                                        [["EmployeeID", "AgencyID", "DateID", "PayBasisID", "FiscalYear", \
                                                        "PayrollNumber", "BaseSalary", "RegularHours", "OTHours", \
                                                        "RegularGrossPaid", "TotalOTPaid", "TotalOtherPay"]]

factpayroll_df.head()

,EmployeeID,AgencyID,DateID,PayBasisID,FiscalYear,PayrollNumber,BaseSalary,RegularHours,OTHours,RegularGrossPaid,TotalOTPaid,TotalOtherPay
0,10001,2120,20200000,1,2020,17,86005.0,1820.0,0.0,84698.21,0.0,0.00
1,149612,2120,20200000,1,2020,17,86005.0,1820.0,0.0,84698.21,0.0,0.00
2,206583,2120,20200000,1,2020,17,86005.0,1820.0,0.0,84698.21,0.0,0.00
3,199874,2120,20200000,1,2020,17,86005.0,1820.0,0.0,87900.95,0.0,-3202.74
4,58036,2120,20200000,1,2020,17,86005.0,1820.0,0.0,83976.54,0.0,0.00


In [13]:
print(factpayroll_df.head())  # Before adding FactID and reordering
print(f"Number of rows in factpayroll_df: {len(factpayroll_df)}")

   EmployeeID  AgencyID    DateID  PayBasisID  FiscalYear  PayrollNumber  \
0       10001      2120  20200000           1        2020             17   
1      149612      2120  20200000           1        2020             17   
2      206583      2120  20200000           1        2020             17   
3      199874      2120  20200000           1        2020             17   
4       58036      2120  20200000           1        2020             17   

   BaseSalary  RegularHours  OTHours  RegularGrossPaid  TotalOTPaid  \
0     86005.0        1820.0      0.0          84698.21          0.0   
1     86005.0        1820.0      0.0          84698.21          0.0   
2     86005.0        1820.0      0.0          84698.21          0.0   
3     86005.0        1820.0      0.0          87900.95          0.0   
4     86005.0        1820.0      0.0          83976.54          0.0   

   TotalOtherPay  
0           0.00  
1           0.00  
2           0.00  
3       -3202.74  
4           0.00  
Nu

In [14]:

           

# Creating the FactID after the merging
factpayroll_df["FactID"] = range(1, len(factpayroll_df) + 1)

# Reorder columns to have FactID first
factpayroll_df = factpayroll_df[[
                "FactID", "EmployeeID", "AgencyID", "DateID", "PayBasisID", "FiscalYear", \
                "PayrollNumber", "BaseSalary", "RegularHours", "OTHours", "RegularGrossPaid", \
                "TotalOTPaid", "TotalOtherPay"
            ]]

In [15]:
print(factpayroll_df.head())  # Before adding FactID and reordering
print(f"Number of rows in factpayroll_df: {len(factpayroll_df)}")

   FactID  EmployeeID  AgencyID    DateID  PayBasisID  FiscalYear  \
0       1       10001      2120  20200000           1        2020   
1       2      149612      2120  20200000           1        2020   
2       3      206583      2120  20200000           1        2020   
3       4      199874      2120  20200000           1        2020   
4       5       58036      2120  20200000           1        2020   

   PayrollNumber  BaseSalary  RegularHours  OTHours  RegularGrossPaid  \
0             17     86005.0        1820.0      0.0          84698.21   
1             17     86005.0        1820.0      0.0          84698.21   
2             17     86005.0        1820.0      0.0          84698.21   
3             17     86005.0        1820.0      0.0          87900.95   
4             17     86005.0        1820.0      0.0          83976.54   

   TotalOTPaid  TotalOtherPay  
0          0.0           0.00  
1          0.0           0.00  
2          0.0           0.00  
3          0.0    

In [ ]:
factpayroll_df.to_sql("FactPayroll", con=conn, if_exists="replace", index=False)
logging.info('FactPayroll data loaded successfully!!')


In [5]:

print(factpayroll_df.head())  # Before adding FactID and reordering
print(f"Number of rows in factpayroll_df: {len(factpayroll_df)}")

NameError: name 'factpayroll_df' is not defined